In [1]:
import datetime
import pandas as pd
import numpy as np
import regex as re
from datetime import time
from functools import reduce
import json
from meteostat import Daily,Hourly
import arrow
from google.cloud import bigquery
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',100)

In [3]:
# read the csv file and name it as "data_rd"

# ============================================================
# ============================================================
data_rd=pd.read_csv('D:\Thermosphr\KPI_task\\main_data.csv')



# ============================================================
# ============================================================
data_rd['value'] = data_rd['value'].astype('float')
data_rd['time'] = data_rd['time_local']
data_rd['time'] = pd.to_datetime(data_rd['time'], format='%Y-%m-%d %H:%M:%S')
dff2 = data_rd[['time', 'alias','value']].copy()
dff2 = dff2[dff2.alias == 'air_temperature']
dff2.set_index('time',inplace=True)
dff2 = dff2.resample('1h').first()

dff2['alias'] = 'air_temperature_1h'
dff2.reset_index(inplace=True)

dff2['time_local'] = dff2['time']

frames = [data_rd, dff2[['time_local', 'time', 'alias', 'value']]]
data_rd = pd.concat(frames, axis=0).reset_index(drop=True)
if data_rd.shape[0] > 0:
    left_table = data_rd.drop(['alias','value','time_local'],axis=1)
    left_table = left_table.groupby("time").min().reset_index()
    sub_df = data_rd[['time','alias','value']].copy()
    sub_df['value'] = pd.to_numeric(sub_df['value'])
    right_table = pd.pivot_table(sub_df,index='time',columns='alias',values='value').reset_index()
    data_rd = pd.merge(left_table,right_table,on='time', how='outer')

main_data = data_rd

In [4]:
data_rd

,time,ahu_10_air_discharge_flow_sensor,ahu_10_air_discharge_flow_sp,ahu_10_air_discharge_temp_sensor,ahu_10_air_discharge_temp_sp,ahu_10_air_return_flow_sensor,ahu_10_air_return_flow_sp,ahu_10_air_return_temp_sensor,ahu_10_chilled_water_valve_sp,ahu_10_co2_return_sensor,ahu_10_enable_sched,ahu_10_fan_air_discharge_enable_cmd,ahu_10_fan_air_return_enable_cmd,ahu_10_heatRecovery_water_valve_sp,ahu_10_hot_water_valve_sp,ahu_11_air_discharge_flow_sp,ahu_11_air_discharge_temp_sensor,ahu_11_air_discharge_temp_sp,ahu_11_air_return_flow_sp,ahu_11_air_return_temp_sensor,ahu_11_chilled_water_valve_sp,ahu_11_co2_return_sensor,ahu_11_enable_sched,ahu_11_fan_air_discharge_enable_cmd,ahu_11_fan_air_return_enable_cmd,ahu_11_heatExchanger_cold_air_entering_temp_sensor,ahu_11_heatRecovery_water_valve_sp,ahu_11_hot_water_valve_sp,ahu_12_air_discharge_flow_sp,ahu_12_air_discharge_temp_sensor,ahu_12_air_discharge_temp_sp,ahu_12_air_return_flow_sp,ahu_12_air_return_temp_sensor,ahu_12_chilled_water_valve_sp,ahu_12_co2_return_sensor,ahu_12_enable_sched,ahu_12_fan_air_discharge_enable_cmd,ahu_12_fan_air_return_enable_cmd,ahu_12_heatExchanger_cold_air_entering_temp_sensor,ahu_12_heatRecovery_water_valve_sp,ahu_12_hot_water_valve_sp,ahu_12_outside_air_damper_cmd,ahu_1_air_discharge_flow_sp,ahu_1_air_discharge_temp_sensor,ahu_1_air_discharge_temp_sp,ahu_1_air_return_flow_sp,ahu_1_air_return_temp_sensor,ahu_1_chilled_water_valve_sp,ahu_1_co2_return_sensor,ahu_1_enable_sched,...,ahu_7_heatExchanger_cold_air_entering_temp_sensor,ahu_7_hot_water_valve_sp,ahu_7_mixed_air_damper_sp,ahu_7_outside_air_damper_sp,ahu_8_air_discharge_temp_sensor,ahu_8_air_discharge_temp_sp,ahu_8_air_return_temp_sensor,ahu_8_enable_sched,ahu_8_hot_water_valve_sp,air_temperature,air_temperature_1h,cloud_cover,dew_point,dhi,dni,elevation,ghi,heatExchanger_1_primaryLoop_hot_water_leaving_temp_sensor,heatExchanger_1_primaryLoop_hot_water_valve_sp,heatExchanger_1_secondaryLoop_hot_water_leaving_temp_sensor,heatExchanger_1_secondaryLoop_hot_water_leaving_temp_sp,heatExchanger_1a_pump_hot_water_enable_cmd,heatExchanger_1a_pump_hot_water_flow_sp,heatExchanger_1b_pump_hot_water_enable_cmd,heatExchanger_1b_pump_hot_water_flow_sp,heatExchanger_2_primaryLoop_hot_water_leaving_temp_sensor,heatExchanger_2_primaryLoop_hot_water_valve_sp,heatExchanger_2_secondaryLoop_hot_water_leaving_temp_sensor,heatExchanger_2_secondaryLoop_hot_water_leaving_temp_sp,heatExchanger_2a_pump_hot_water_enable_cmd,heatExchanger_2a_pump_hot_water_flow_sp,heatExchanger_2b_pump_hot_water_enable_cmd,heatExchanger_2b_pump_hot_water_flow_sp,heatExchanger_primaryLoop_hot_water_entering_temp_sensor,heatExchanger_primaryLoop_hot_water_leaving_temp_sp,humidity,ice_cover,lat,lng,plant_districtHeating_outside_air_temp_sensor,precipitation,pressure,snow_depths,sp_apparent_zenith,sp_azimuth,sp_zenith,specific_humidity,visibility,wet_bulb_temperature,wind_gust
0,2022-09-30 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.5900,7.59,91.6000,3.854668,0.0,0.0,194.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.2000,0.0,50.875698,12.077743,NaN,0.00,1008.1100,0.0,130.300380,19.193106,130.300380,0.004981,24.13,5.775341,0.7000
1,2022-09-30 02:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.6325,NaN,91.8325,3.869543,0.0,0.0,194.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.0575,0.0,50.875698,12.077743,NaN,0.00,1008.1600,0.0,129.434165,23.942860,129.434165,0.004986,24.13,5.803105,0.7075
2,2022-09-30 02:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [5]:
class kpi():
    def __init__(self, rule, batch):

        # get subset of data, min-time and max-time for the given start and end time
        batch, batch_min_time, batch_max_time = self.__batch_subset(batch, rule['start_time'], rule['end_time'])
 
        # indices of the rule dataframe
        alias_cols = rule.index
        subs = 'alias'
        # get columns of rule book that alias in their names
        self.__res = [alias_col for alias_col in alias_cols if subs in alias_col]
        self.__rule = rule
        
        # default logs columns
        log_columns = ['kpi_name', 'time', 'aliases', 'value', 'aggregation']
        for col in self.__res:
            log_columns.append(col)

        # create empty dataframe for the logs
        self.__df_logs = pd.DataFrame(columns=['kpi_name', 'time', 'aliases', 'value', 'aggregation'])
        aliases = []
        # get a list of aliases for the given targets in the rule book and append that lists
        # in the aliases list
        for i in self.__res:
            if rule[i] is not None:
                aliases.append(self.__alias_substrings(batch, [rule[i]], col_name='alias'))
        # in case aliases list length is 1
        # create logs for only one target (alias_a)
        if len(aliases) == 1:
            for alias in aliases[0]:
                # get a dataframe for each alias
                df = self.alias_dataframe(self, data_rd=batch, aliases=alias, kpi=rule['kpi_name'],
                                          resample_grain=None, daily_resample=None, agg=None, non_negative=None)
                # apply the equation that is given in the rule book (equation_1)
                df = self.apply_equation(self, df, equation=rule['equation_1'], temp=True)
                # Aggregate the data after applying equation_1
                df = self.apply_aggregation(self, df, aggregation=rule['aggregation'], kpi=rule['kpi_name'])
                # In case equation_2 is given then apply it after aggregation
                if rule['equation_2'] is not None:
                    df = self.apply_equation(self, df, equation=rule['equation_2'])

                df1 = self.__logs_data(df, rule, [alias])
                self.__df_logs = pd.concat([self.__df_logs, df1], axis=0)

        else:
            alias_pair = self.__alias_pairs(self, batch, rule, aliases)

            for pair in alias_pair:
                if len(pair) != 0:

                    df = self.alias_pairs_dataframe(self, data_rd=batch, aliases=pair, kpi=rule['kpi_name'],
                                                    resample_grain=None, daily_resample=None, agg=None,
                                                    non_negative=None)
                    df = self.apply_equation(self, df, equation=rule['equation_1'], temp=True)
                    df = self.apply_aggregation(self, df, aggregation=rule['aggregation'], kpi=rule['kpi_name'])
                    # In case equation_2 is given then apply it after aggregation
                    if rule['equation_2'] is not None:
                        df = self.apply_equation(self, df, equation=rule['equation_2'])

#                     display(df)
                    df1 = self.__logs_data(df, rule, pair)
                    self.__df_logs = pd.concat([self.__df_logs, df1], axis=0)
    


    @staticmethod
    def __logs_data(df=None, rule=None, alias_list=None):
        """ Gets resultant dataframe and rules for each kpi and creates logs
            in the form of a dataframe.
        Parameters
        ----------
        df: dataframe
        Resultant dataframe for each kpi
        rule:dataframe
        Its the config file
        tgt_alias:string
        Name of the target alias
        ref_alias: string
        Name of the reference alias
        """
        df1 = pd.DataFrame()

        if len(alias_list) == 1:
            logs_alias = alias_list[0]
        else:
            logs_alias = ",".join(alias_list)

        new_row = {'kpi_name': rule['kpi_name'],
                   'time': df['time'],
                   'aliases': logs_alias,
                   'value': df['value'],
                   'equation_1': rule['equation_1'],
                   'equation_2': rule['equation_2'],
                   'aggregation': rule['aggregation'],
                   }

        df1 = pd.DataFrame(new_row)
        if rule.mute_alert is not None:
            for x in rule.mute_alert[0].split(','):
                df1['mute_alert'] = np.where(df1.aliases.str.contains(x), True, False)
        else:
            df1['mute_alert'] = False
        return df1

    @staticmethod
    def apply_equation(self, df=None, equation=None, temp=None):

        def func(x):
            y = eval(equation)
            return y

        if df.shape[0] > 0:
            if equation is not None:

                for col in self.__res:
                    print(col)
                    if col in equation:
                        col_name = f"x['{col}']"
                        equation = equation.replace(col, col_name)
                if 'values' not in equation:
                    if 'value' in equation:
                        equation = equation.replace('value',"x['value']")

                if 'time_local' in equation:
                    df['time_local'] = pd.to_datetime(df['time_local'])
                    df['time_only'] = df['time_local'].dt.time.astype('str')
                    equation = equation.replace('time_local', 'x["time_only"]')
                    time_list = re.findall(r'\d{2}\:\d{2}\:\d{2}', equation)
                    for tt in time_list:
                        time_split = tt.split(":")
                        updated_time = time(int(time_split[0]), int(time_split[1]), int(time_split[2]))
                        equation = re.sub(tt, str(updated_time), equation)
                df['value'] = df.apply(func, axis=1)
#                 df['value'] = eval(equation)

            else:
                df['value'] = df['alias_a']

        return df


    @staticmethod
    def apply_aggregation(self, df=None, aggregation='mean', kpi=None):

        if df.shape[0] > 0:
            if aggregation == 'perc':
                aggregation = 'mean'

            df.set_index('time', inplace=True)

            x = f"df.resample('D').{aggregation}()"

            df = eval(x)

            df.reset_index(inplace=True)

            df['kpi'] = kpi

        df.loc[df["value"] == True, "value"] = 1
        df.loc[df["value"] == False, "value"] = 0

        return df

    @staticmethod
    def __alias_substrings(df, in_list, col_name='alias'):
        """ This function returns a list of aliases for a given reference in the
            the config file.
        Parameters
        ----------
        df: dataframe
        It's the source dataframe
        in_list: list
        It's the list of given aliases in the config file
        col_name: string
        It's the column name in the source dataframe
        Returns
        -------
        filtered_aliases: list
        This list contains the aliases that are filtered out
        """
        groups = in_list

        substrings = []
        filtered_aliases = []
        # Creating list of substrings
        for group_id in groups:
#             print("group_id ==> ", group_id)
#             print("group_id data type ==> ",type(group_id))
            substrings.append(group_id.split('_'))

        unique_aliases = df.columns
        # print(unique_aliases)
        # Removing None from the list of unique aliases
        unique_aliases = list(filter(None, unique_aliases))
        # Filter the aliases that are to be used for the KPI
        for alias in unique_aliases:
            for sub in substrings:
                if all(a in alias for a in sub):
                    filtered_aliases.append(alias)

        return filtered_aliases

    @staticmethod
    def __batch_subset(batch, start_time, end_time):
        """Gets a dataframe and a specific duration, where it takes a subset
        of dataframe that falls in the start and end times.
        Parameters
        ----------
        batch: dataframe
        The source dataframe
        start_time: timestamp
        This will be the start or minimum time of the dataframe
        end_time: timestamp
        This be the end or maximum time of the dataframe
        Returns
        ------
        batch: dataframe
        This is resultant dataframe between the start and end times
        batch_min_time: timestamp
        The minimum time of the dataframe
        batch_max_time: timestamp
        The maximum time of the dataframe
        """
        # in case start time is given
        if start_time is not None:
            # start_time = str(start_time)
            # split the time(string) in hour,min and secs
            hour = start_time.split(':')[0]
            minute = start_time.split(':')[1]
            second = start_time.split(':')[2]
            print(f'start_time {time(int(hour), int(minute), int(second))}')
            # filter dataframe having time  greater or equal  to start time
            batch = batch.loc[batch['time'].dt.time >= time(int(hour), int(minute), int(second))]
        # in case end time is given
        if end_time is not None:
            # end_time = str(end_time)
            # split the time(string) in hour,min and secs
            hour = end_time.split(':')[0]
            minute = end_time.split(':')[1]
            second = end_time.split(':')[2]
            print(f"end_time {time(int(hour), int(minute), int(second))}")
            # filter dataframe having time  smaller than the  end time
            batch = batch.loc[batch['time'].dt.time < time(int(hour), int(minute), int(second))]
        # get miminium and maximum time of the dataframe
        batch_min_time = batch.time.min()
        batch_max_time = batch.time.max()
        return batch, batch_min_time, batch_max_time

    @staticmethod
    def alias_pairs_dataframe(self, data_rd=None, aliases=None, kpi=None, resample_grain=None,
                              daily_resample=None, agg=None, non_negative=None):
        df_master = pd.DataFrame()
        res = [index for index, val in enumerate(aliases)]
        alias_cols = aliases.copy()
        alias_cols.append('time')
        cols = [val for index, val in enumerate(self.__res) if index in res]
        cols.append('time')
        df = data_rd[alias_cols].copy()
        df.columns = cols
        if df.shape[0] > 0:
            df_master = df
        return df_master

    @staticmethod
    def alias_dataframe(self, data_rd=None, aliases=None, kpi=None, resample_grain=None,
                        daily_resample=None, agg=None, non_negative=None):
        df_master = pd.DataFrame()
        df = data_rd[['time', aliases]].copy()
        df.rename(columns={aliases: 'alias_a'}, inplace=True)
        df_master = df

        return df_master

    @staticmethod
    def __alias_pairs(self, data_rd, rule, aliases):

        self.__index = [index for index, val in enumerate(aliases)]
        col_names = self.__res
        col_names = [val for index, val in enumerate(col_names) if index in self.__index]
        null_list_index = [index for index, val in enumerate(aliases) if len(val) == 0]
        non_null_dfs_list = []
        null_dfs_list = []

        for i in range(len(aliases)):

            non_null_alias_df = pd.DataFrame(columns=['alias', 'keys'])
            null_alias_df = pd.DataFrame(columns=['alias', 'keys'])
            if len(aliases[i]) > 0:
                for alias in aliases[i]:
                    keys = alias.split("_")
                    for ele in rule[col_names[i]].split('_'):
                        keys.remove(ele)

                    if len(keys) == 0:

                        df = pd.DataFrame([[alias, None]], columns=['alias', 'keys'])

                        null_alias_df = pd.concat([null_alias_df, df], axis=0, ignore_index=True)


                    else:

                        key = '_'.join(map(str, keys))
                        df = pd.DataFrame([[alias, key]], columns=['alias', 'keys'])
                        non_null_alias_df = pd.concat([non_null_alias_df, df], axis=0, ignore_index=True)

            if non_null_alias_df.shape[0] > 0:
                non_null_dfs_list.append(non_null_alias_df)
            if null_alias_df.shape[0] > 0:
                null_dfs_list.append(null_alias_df)

        if len(non_null_dfs_list) > 1:
            non_nulls_df_merge = reduce(lambda left, right: pd.merge(left, right, on=["keys"], how="inner"),
                                        non_null_dfs_list)
        elif len(non_null_dfs_list) == 1:
            non_nulls_df_merge = non_null_dfs_list[0]
        else:
            non_nulls_df_merge = pd.DataFrame(columns=['alias', 'keys'])

        if len(null_dfs_list) > 1:
            nulls_df_merge = reduce(lambda left, right: pd.merge(left, right, how="cross"), null_dfs_list)
        elif len(null_dfs_list) == 1:
            nulls_df_merge = null_dfs_list[0]
        else:
            nulls_df_merge = pd.DataFrame(columns=['alias', 'keys'])

        if nulls_df_merge.shape[0] > 0 and non_nulls_df_merge.shape[0] > 0:
            alias_pairs_df = pd.merge(nulls_df_merge, non_nulls_df_merge, how='cross')
        elif nulls_df_merge.shape[0] == 0:
            alias_pairs_df = non_nulls_df_merge
        elif non_nulls_df_merge.shape[0] == 0:
            alias_pairs_df = nulls_df_merge

        subs = 'keys'
        keys_cols = [col for col in alias_pairs_df if subs in col]
        alias_pairs_df = alias_pairs_df.drop(keys_cols, axis=1)

        # display(alias_pairs_df)

        if len(null_list_index) == 0:
            alias_pairs = alias_pairs_df.values.tolist()
        else:
            alias_pairs = []

        return alias_pairs

    def return_log(self):
        return self.__df_logs

In [6]:
with open('D:\Thermosphr\KPI_task\payload.json') as file:
    config = json.loads(file.read())
    file.close()
station_id = config['settings']['station_id']
print('station id ==> ',station_id)
df = pd.DataFrame(config['rules'])
df

KeyError: 'settings'

In [39]:
logs = pd.DataFrame()

min_time = data_rd.time.min()
max_time = data_rd.time.max()
print(min_time)
print(max_time)

def station_data(station_id,start,end):
    data = Daily(station_id,start,end)
    data = data.fetch()
    data.reset_index(inplace=True)
    data = data[['time','tavg','tmin','tmax']]
    return data
    
if station_id:
    meteo_data = station_data(station_id,min_time,max_time)
    data_rd = data_rd.merge(meteo_data,on='time',how='left')
    data_rd[['tavg','tmax','tmin']] = data_rd[['tavg','tmax','tmin']].astype('float')



if data_rd.shape[0] > 0:
    for i in range(df.shape[0]):
        obj = kpi(df.iloc[i], data_rd)
        dff = obj.return_log()
        logs = pd.concat([logs, dff], axis=0)

        if df.iloc[i]['mute_logs']:
            aliases_list = df.iloc[i]['mute_logs'].split(',')
            for alias in aliases_list:
                logs = logs[~logs.aliases.str.contains(alias.strip())]
else:
    print(f"Data is not available from {start_time} to {end_time}")
logs

2022-12-13 00:00:00
2022-12-13 23:45:00


,time,tavg,tmin,tmax
0,2022-12-13,-1.7,-3.9,1.9
